# Coursera Capstone Week 4

## Final Project

#### 0.1 Import Libraries

In [1]:
import folium # map rendering library
import pandas as pd
import numpy as np
import requests
import geocoder
import time
import re
import os.path

from geopy import distance
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from tqdm import tqdm_notebook

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

#### 0.2 Define FourSquare client id and secret

In [2]:
CLIENT_ID = 'IHQ1K1BCOLXGHJA2V3TFHOWQWXIP15ZVDRQI2CVTU0TI1DFF' # your Foursquare ID
CLIENT_SECRET = '2UBYTLDBVPFJOKSWSIHDS4VHEJSCQ1R1KMRB2HAY1QICHEXL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IHQ1K1BCOLXGHJA2V3TFHOWQWXIP15ZVDRQI2CVTU0TI1DFF
CLIENT_SECRET:2UBYTLDBVPFJOKSWSIHDS4VHEJSCQ1R1KMRB2HAY1QICHEXL


#### 0.3 Set row limit and radii for FourSquare

In [3]:
LIMIT = 300 # limit of number of venues returned by Foursquare API

### Functions

#### 1.0 Define function to build table of PostCodes

In [4]:
def populateTable(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    data = []
    table = soup.find('table', attrs={'class':'wikitable sortable'})
    table_body = table.find('tbody')

    rows = table_body.find_all(re.compile('tr'))
    for row in rows:
        cols = row.find_all(re.compile('t[h|d]'))
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values
        
    dfPostCodes = pd.DataFrame(data)
    #dfPostCodes.columns = ['PostalCode','Town','Coverage','Local Authority']
    dfPostCodes.columns = dfPostCodes.iloc[0]
    dfPostCodes = dfPostCodes[dfPostCodes['Coverage'] != 'Coverage'].reset_index(drop=True)
    dfPostCodes = dfPostCodes[dfPostCodes['Coverage'] != 'non-geographic'].reset_index(drop=True)
    dfPostCodes = dfPostCodes[dfPostCodes['Local authority area'] != 'non-geographic'].reset_index(drop=True)
    
    return dfPostCodes

#### 1.1 Define function to get category type from venue

In [5]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### 1.2 Define function to obtain closest postcode to venue coordinates

In [6]:
def getVenuePostcode(lat, lng):
    url = 'https://api.postcodes.io/postcodes?lon={}&lat={}'.format(lng,lat)
    results = requests.get(url).json()
    return results['result'][0]['postcode'], results['result'][0]['outcode']

#### 1.3 Define function to get nearby venues

In [7]:
def getNearbyVenues(names, latitudes, longitudes, postcodes, radius=1500, limit=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            print(url)
            break
        
        # return only relevant information for each nearby venue
        for v in tqdm_notebook(results, leave=False):
            venueName = v['venue']['name']
            venueLat = v['venue']['location']['lat']
            venueLng = v['venue']['location']['lng']
            venueCategory = v['venue']['categories'][0]['name']
            try:
                venuePostcode, venueOutcode = getVenuePostcode(venueLat,venueLng)
                venuePostalRegion = postcodes[venuePostcode.split(' ')[0]]
            except:
                venuePostcode = ''
                venuePostalRegion = ''
            
            try:
                venues_list.append([(
                    name, 
                    lat, 
                    lng, 
                    venueName, 
                    venueLat, 
                    venueLng,  
                    venueCategory,
                    venuePostcode,
                    venueOutcode,
                    venuePostalRegion)])
            except:
                print(v['venue']['name'])
                continue
                
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood Postcode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue Postcode',
                  'Venue Postal Outcode',
                  'Venue Postal Region']
    
    return(nearby_venues)

#### 1.4 Define function to obtain most common venue categories

In [8]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Build postcode and venue datasets

#### 2.0 Loop through Postal Districts and populate DataFrames with PostCodes

In [9]:
postalDistricts = ['E','EC','N','NW','SE','SW','W','WC']
pages = dict()
tables = dict()
dictPostcodes = dict()

for pD in tqdm_notebook(postalDistricts):
    pages[pD] = 'https://en.wikipedia.org/wiki/{}_postcode_area'.format(pD)
    
    if not(os.path.isfile(pD + '.csv')):
        tables[pD] = populateTable(pages[pD])

        latitude = []
        longitude = []

        n=tables[pD].shape[0]
        postal_code = tables[pD]['Postcode district'].values

        for i in tqdm_notebook(range(n)):

            # initialize your variable to None

            lat_lng_coords = None

            # loop until you get the coordinates

            j = 0
            while(lat_lng_coords is None):
                j += 1
                g = geocoder.google('{}, London, England'.format(postal_code[i]))

                lat_lng_coords = g.latlng

            latitude.append(lat_lng_coords[0])
            longitude.append(lat_lng_coords[1])
        tables[pD]['Latitude'] = latitude
        tables[pD]['Longitude'] = longitude
        tables[pD]['Postal Region'] = pD

        #dfSave = tables[pD][['Postcode district','Latitude','Longitude']]
        tables[pD].to_csv(pD + '.csv',index=False)
    else:
        tables[pD] = pd.read_csv(pD + '.csv')
        
    tables[pD]['Postal Region'] = pD
    fixed_columns = [tables[pD].columns[-1]] + list(tables[pD].columns[:-1])
    tables[pD] = tables[pD][fixed_columns]

for pD in postalDistricts:
    tables[pD]=tables[pD].rename(columns = {'Postcode district':'Postcode'})
    for index, row in tables[pD].iterrows():
        dictPostcodes[row['Postcode']] = row['Postal Region']

#### 2.1 Show a subsample of postcode data

In [10]:
tables['E'].head()

,Postal Region,Postcode,Post town,Coverage,Local authority area,Latitude,Longitude
0,E,E1,LONDON,"Eastern head district: Aldgate (part), Bishops...","Tower Hamlets, Hackney, City of London",51.517623,-0.055755
1,E,E1W,LONDON,"Wapping, St Katharine Docks, Stepney (part), S...",Tower Hamlets,51.505796,-0.057199
2,E,E2,LONDON,Bethnal Green district: Bethnal Green (mostly)...,"Tower Hamlets, Hackney",51.528134,-0.061651
3,E,E3,LONDON,"Bow district: Bow, Bow Common, Bromley-by-Bow,...","Tower Hamlets, Newham",51.528148,-0.020542
4,E,E4,LONDON,"Chingford district: Chingford, Sewardstone, Hi...","Waltham Forest, Enfield, Epping Forest (Essex)",51.630750,-0.011780


#### 2.2 Loop though postal districts and retrieve venues within 1Km

In [11]:
nearbyBusinesses = dict()
   
for pD in tqdm_notebook(postalDistricts):
    if not(os.path.isfile('NB.' + pD + '.csv')):
        nearbyBusinesses[pD] = getNearbyVenues(names=tables[pD]['Postcode'],
                                             latitudes=tables[pD]['Latitude'],
                                             longitudes=tables[pD]['Longitude'],
                                             postcodes=dictPostcodes,
                                             radius = 1000,
                                             limit=3000)

        nearbyBusinesses[pD].to_csv('NB.' + pD + '.csv',index=False)
    else:
        print(pD + ' exists...')
        nearbyBusinesses[pD] = pd.read_csv('NB.' + pD + '.csv')

    

E exists...
EC exists...
N exists...
NW exists...
SE exists...
SW exists...
W exists...
WC exists...



#### 2.3 Show a subsample of venue data

In [12]:
nearbyBusinesses['E'].head()

,Neighborhood Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Postcode,Venue Postal Outcode,Venue Postal Region
0,E1,51.517623,-0.055755,Mouse Tail Coffee Stories,51.519471,-0.058573,Coffee Shop,E1 5DG,E1,E
1,E1,51.517623,-0.055755,Rinkoff's Bakery,51.519964,-0.053238,Bakery,E1 3AR,E1,E
2,E1,51.517623,-0.055755,Genesis Cinema,51.521036,-0.051073,Movie Theater,E1 4UH,E1,E
3,E1,51.517623,-0.055755,The White Hart,51.520245,-0.056385,Pub,E1 5QH,E1,E
4,E1,51.517623,-0.055755,Dirty Burger/Chicken Shop,51.520253,-0.055112,Burger Joint,E1 4TL,E1,E


#### 2.4 Let's check number of venues per postal district

In [13]:
for pD in postalDistricts:
    print('Nearby ' + pD + ' ... ' + str(nearbyBusinesses[pD].shape[0]))

Nearby E ... 1392
Nearby EC ... 2800
Nearby N ... 1289
Nearby NW ... 616
Nearby SE ... 1480
Nearby SW ... 2230
Nearby W ... 2293
Nearby WC ... 1400


#### 2.5 Concatenate our venues and postcodes dataframes

In [14]:
vCols = ['Venue','Venue Latitude','Venue Longitude','Venue Category','Venue Postcode','Venue Postal Outcode','Venue Postal Region']
pCols = ['Postal Region','Postcode','Post town','Coverage','Local authority area','Latitude','Longitude']

london_venues = pd.DataFrame(columns=vCols)
for pD in postalDistricts:
    london_venues = london_venues.append(nearbyBusinesses[pD][vCols])
london_venues.drop_duplicates(inplace=True)
london_venues.reset_index(inplace=True, drop=True)
print(london_venues.shape)

london_postcodes = pd.DataFrame(columns=pCols)
for pD in postalDistricts:
    london_postcodes = london_postcodes.append(tables[pD][pCols])
london_postcodes.drop_duplicates(inplace=True)
london_postcodes.reset_index(inplace=True, drop=True)
print(london_postcodes.shape)


(8294, 7)
(178, 7)


#### 2.6 Show first 5 records of london_venues

In [15]:
london_venues.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Postcode,Venue Postal Outcode,Venue Postal Region
0,Mouse Tail Coffee Stories,51.519471,-0.058573,Coffee Shop,E1 5DG,E1,E
1,Rinkoff's Bakery,51.519964,-0.053238,Bakery,E1 3AR,E1,E
2,Genesis Cinema,51.521036,-0.051073,Movie Theater,E1 4UH,E1,E
3,The White Hart,51.520245,-0.056385,Pub,E1 5QH,E1,E
4,Dirty Burger/Chicken Shop,51.520253,-0.055112,Burger Joint,E1 4TL,E1,E


#### 2.7 Show first 5 records of london_postcodes

In [16]:
london_postcodes.head()

,Postal Region,Postcode,Post town,Coverage,Local authority area,Latitude,Longitude
0,E,E1,LONDON,"Eastern head district: Aldgate (part), Bishops...","Tower Hamlets, Hackney, City of London",51.517623,-0.055755
1,E,E1W,LONDON,"Wapping, St Katharine Docks, Stepney (part), S...",Tower Hamlets,51.505796,-0.057199
2,E,E2,LONDON,Bethnal Green district: Bethnal Green (mostly)...,"Tower Hamlets, Hackney",51.528134,-0.061651
3,E,E3,LONDON,"Bow district: Bow, Bow Common, Bromley-by-Bow,...","Tower Hamlets, Newham",51.528148,-0.020542
4,E,E4,LONDON,"Chingford district: Chingford, Sewardstone, Hi...","Waltham Forest, Enfield, Epping Forest (Essex)",51.630750,-0.011780


#### 2.8 Loop through the venue and postcode DataFrames and calculate the distance between the two

In [17]:
if not(os.path.isfile('london_venues.csv')):
    for index, row in london_postcodes.iterrows():
        london_venues[row['Postcode']] = 0.0000

    with tqdm_notebook(total=len(list(london_venues.iterrows()))) as pbar: 
        for index, row in london_venues.iterrows():
            pbar.update(1)
            coords1 = (row['Venue Latitude'],row['Venue Longitude'])
            for index2, row2 in london_postcodes.iterrows():
                coords2 = (row2['Latitude'],row2['Longitude'])
                london_venues.loc[index, row2[['Postcode']]] = distance.geodesic(coords1, coords2).m
    
        #print(london_venues.loc[0, 'E1'])
    london_venues.to_csv('london_venues.csv',index=False)
else:
    print('London Venues exists...')
    london_venues = pd.read_csv('london_venues.csv')
    

London Venues exists...


#### 2.9 Check the contents of london_venues

In [18]:
london_venues.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Postcode,Venue Postal Outcode,Venue Postal Region,E1,E1W,E2,E3,E4,E5,E6,E7,E8,E9,E10,E11,E12,E13,E14,E15,E16,E17,E18,E20,E77,E98,EC1A,EC1M,EC1N,EC1P,EC1R,EC1V,EC1Y,EC2A,EC2M,EC2N,EC2P,EC2R,EC2V,EC2Y,EC3A,EC3M,EC3N,EC3P,EC3R,EC3V,EC4A,EC4M,EC4N,EC4P,EC4R,EC4V,EC4Y,EC50,N1,N1C,N2,N3,N4,N5,N6,N7,N8,N9,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20,N21,N22,NW1,NW2,NW3,NW4,NW5,NW6,NW7,NW8,NW9,NW10,NW11,SE1,SE2,SE3,SE4,SE5,SE6,SE7,SE8,SE9,SE10,SE11,SE12,SE13,SE14,SE15,SE16,SE17,SE18,SE19,SE20,SE21,SE22,SE23,SE24,SE25,SE26,SE27,SE28,SW1A,SW1E,SW1H,SW1P,SW1V,SW1W,SW1X,SW1Y,SW2,SW3,SW4,SW5,SW6,SW7,SW8,SW9,SW10,SW11,SW12,SW13,SW14,SW15,SW16,SW17,SW18,SW19,SW20,W1B,W1C,W1D,W1F,W1G,W1H,W1J,W1K,W1S,W1T,W1U,W1W,W2,W3,W4,W5,W6,W7,W8,W9,W10,W11,W12,W13,W14,WC1A,WC1B,WC1E,WC1H,WC1N,WC1R,WC1V,WC1X,WC2A,WC2B,WC2E,WC2H,WC2N,WC2R
0,Mouse Tail Coffee Stories,51.519471,-0.058573,Coffee Shop,E1 5DG,E1,E,283.790058,1524.497955,987.204411,2810.526519,12798.708543,4514.640314,8167.678294,7083.916412,2727.400985,2712.937793,5906.273901,7951.148189,8839.348694,6230.337954,3043.269220,5088.664972,6197.105121,8370.233139,9929.654123,4396.251530,4988.589414,4988.589414,2815.364911,2918.863007,3424.441924,3378.772633,3414.532332,2685.563433,2246.730309,1625.189933,1594.006741,1709.509028,4988.589414,2040.889108,2340.973826,2302.435547,1497.525193,1723.054459,1477.277936,4988.589414,1803.238539,1930.293750,3349.082221,2769.437554,2231.282145,4988.589414,2420.375633,2843.950600,3432.048307,4988.589414,3176.061688,5011.064906,11004.381857,12987.034692,6594.256819,4713.381686,8889.850481,5498.926826,8414.890478,12087.784369,10426.527935,12235.491928,13768.567855,11340.200782,13727.024674,7215.163191,4924.426493,8288.818425,10693.864269,7193.967803,16025.731547,13279.961881,9950.715626,6258.025593,12783.139042,8560.560796,13806.895638,7091.350433,9984.233887,15930.623667,8118.772300,15260.602066,13710.057850,11792.971232,2983.944600,12962.416312,7817.964925,6883.244311,5375.472183,9692.306589,7671.010519,4844.356802,11897.236606,5778.725018,4674.537682,9669.680592,7835.143001,5085.232967,5471.526238,2615.509706,4272.549659,10425.077740,11457.287249,11895.081808,9159.264266,7292.698364,8410.933904,7692.613806,13682.795642,10143.355903,10592.475568,11769.103506,5322.635462,6031.067063,5504.434472,5680.785956,6500.196431,6904.959058,7056.489550,5319.539465,8902.998770,7800.963837,8624.555558,9757.828851,11130.716875,8360.187728,7062.570500,6831.656875,9408.609979,9017.158050,10504.590238,13466.802466,15710.016964,13716.146573,12054.473727,12525.998473,12394.420248,14588.693731,17051.622170,5797.855539,6213.637707,4987.219269,5392.987718,6077.968323,7048.189526,6149.437992,6501.913810,5733.937109,5210.477974,6639.715294,5669.596838,7836.584597,14531.357966,14702.786073,16796.355276,12054.090247,19032.430638,9396.856079,8838.936704,10836.248776,10448.476409,11898.723380,18199.052405,10721.716641,4473.280879,4546.830244,4964.814742,4708.915343,4222.969848,3858.383147,4014.463477,3869.682454,3797.490723,4177.274640,4426.860571,4683.207830,4630.269920,4061.101629
1,Rinkoff's Bakery,51.519964,-0.053238,Bakery,E1 3AR,E1,E,313.616195,1600.106846,1080.384607,2445.164954,12656.641756,4425.967678,7794.183622,6738.693555,2727.772391,2517.333844,5675.916589,7667.924360,8483.597940,5856.688756,2722.744912,4738.926346,5843.226220,8198.941850,9669.423426,4075.809364,5360.109493,5360.109493,3188.040593,3286.477224,3794.939874,3750.107622,3773.669459,3026.168469,2601.632868,1965.475631,1967.702729,2082.741633,5360.109493,2414.541706,2715.053163,2673.013502,1862.835010,2078.132124,1797.505901,5360.109493,2131.360815,2295.975932,3723.316051,3143.105227,2596.853519,5360.109493,2781.629069,3213.887818,3804.366334,5360.109493,3388.311131,5333.005118,11223.745025,13214.566016,6715.760078,4889.173455,9120.675696,5730.036140,8573.841930,12033.149650,10597.826717,12359.314456,13954.910315,11394.264444,13802.092298,7234.600104,4978.447068,

### Clustering

#### 3.1 Perform one-hot-encoding of venue data

In [19]:
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")
london_onehot['Venue Postal Outcode'] = london_venues['Venue Postal Outcode']
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Venue Postal Outcode,ATM,Accessories Store,Acupuncturist,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Castle,Caucasian Restaurant,Cemetery,Chaat Place,Champagne Bar,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Cigkofte Place,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Housing Development,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Iraqi Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Observatory,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Gym,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Paintball Field,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pil

#### 3.2 Group data by Postal Region and calculate the frequency of occurance of each category

In [20]:
london_grouped = london_onehot.groupby('Venue Postal Outcode').mean().reset_index()
london_grouped.head()

,Venue Postal Outcode,ATM,Accessories Store,Acupuncturist,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Castle,Caucasian Restaurant,Cemetery,Chaat Place,Champagne Bar,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Cigkofte Place,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Housing Development,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Iraqi Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Observatory,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Gym,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Paintball Field,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pil

#### 3.3 Print each region with it's top 5 most common venues

In [21]:
num_top_venues = 5

for hood in london_grouped['Venue Postal Outcode']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Venue Postal Outcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----CR4----
               venue  freq
0      Train Station   0.2
1  Fish & Chips Shop   0.2
2        Coffee Shop   0.2
3           Platform   0.2
4  Indian Restaurant   0.2


----E1----
         venue  freq
0          Pub  0.09
1  Coffee Shop  0.09
2        Hotel  0.05
3  Pizza Place  0.04
4         Café  0.04


----E10----
           venue  freq
0  Grocery Store  0.18
1           Park  0.16
2           Café  0.08
3          Hotel  0.05
4            Pub  0.05


----E11----
         venue  freq
0          Pub  0.12
1         Café  0.09
2  Coffee Shop  0.07
3       Bakery  0.04
4     Pharmacy  0.04


----E12----
                  venue  freq
0  Gym / Fitness Center  0.22
1         Train Station  0.11
2            Restaurant  0.11
3   Fried Chicken Joint  0.11
4                   Pub  0.11


----E13----
           venue  freq
0  Grocery Store  0.14
1            Pub  0.09
2  Metro Station  0.09
3       Bus Stop  0.09
4           Café  0.05


----E14----
                  venue  freq
0    

4  Peruvian Restaurant   0.0


----KT3----
               venue  freq
0             Bakery  0.33
1           Bus Stop  0.33
2  French Restaurant  0.33
3             Palace  0.00
4           Pharmacy  0.00


----N1----
                venue  freq
0                 Pub  0.20
1                Café  0.10
2         Coffee Shop  0.06
3  Italian Restaurant  0.04
4                Park  0.04


----N10----
                  venue  freq
0                  Café  0.13
1                   Pub  0.10
2           Coffee Shop  0.06
3           Pizza Place  0.05
4  Gym / Fitness Center  0.03


----N11----
               venue  freq
0      Grocery Store  0.12
1           Bus Stop  0.08
2        Coffee Shop  0.08
3               Café  0.04
4  Convenience Store  0.04


----N12----
           venue  freq
0    Coffee Shop  0.15
1           Café  0.10
2  Grocery Store  0.07
3    Supermarket  0.07
4      Bookstore  0.05


----N13----
                  venue  freq
0                   Pub  0.08
1         Grocery 

4         Chinese Restaurant  0.03


----W13----
                  venue  freq
0                  Park  0.10
1                 Hotel  0.08
2         Grocery Store  0.08
3  Fast Food Restaurant  0.08
4     Indian Restaurant  0.05


----W14----
                venue  freq
0                 Pub  0.11
1               Hotel  0.07
2  Italian Restaurant  0.07
3   Indian Restaurant  0.05
4  Persian Restaurant  0.05


----W1A----
                  venue  freq
0            Restaurant  0.33
1      Department Store  0.33
2         Radio Station  0.33
3                   ATM  0.00
4  Pakistani Restaurant  0.00


----W1B----
                           venue  freq
0                 Clothing Store  0.10
1                          Hotel  0.07
2              Indian Restaurant  0.07
3              French Restaurant  0.07
4  Vegetarian / Vegan Restaurant  0.03


----W1C----
            venue  freq
0  Clothing Store  0.18
1  Cosmetics Shop  0.12
2           Hotel  0.12
3    Dessert Shop  0.06
4            

#### 3.4 Now create a DataFrame showing the top 10 venue categories for each region

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Venue Postal Outcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Venue Postal Outcode'] = london_grouped['Venue Postal Outcode']

for ind in np.arange(london_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Venue Postal Outcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,CR4,Train Station,Indian Restaurant,Fish & Chips Shop,Coffee Shop,Platform,Fish Market,Factory,Falafel Restaurant,Farm,Farmers Market
1,E1,Pub,Coffee Shop,Hotel,Indian Restaurant,Pizza Place,Café,Grocery Store,Turkish Restaurant,Park,Thai Restaurant
2,E10,Grocery Store,Park,Café,Coffee Shop,Pub,Hotel,Restaurant,Soccer Stadium,Train Station,Fast Food Restaurant
3,E11,Pub,Café,Coffee Shop,Pharmacy,Bakery,Restaurant,Fish & Chips Shop,Fast Food Restaurant,Metro Station,Supermarket
4,E12,Gym / Fitness Center,Fried Chicken Joint,Train Station,Pub,Indian Restaurant,Restaurant,Park,Turkish Restaurant,Film Studio,Exhibit
5,E13,Grocery Store,Metro Station,Pub,Bus Stop,Hostel,Gym / Fitness Center,Park,Bakery,Pharmacy,Fish & Chips Shop
6,E14,Coffee Shop,Italian Restaurant,Park,Burger Joint,Hotel,Gym / Fitness Center,Plaza,Trail,Sandwich Place,Café
7,E15,Pub,Café,Hotel,Grocery Store,General Entertainment,Bus Stop,Bar,Park,Coffee Shop,Fast Food Restaurant
8,E16,Hotel,Coffee Shop,Chinese Restaurant,Bar,Harbor / Marina,Café,Sandwich Place,Light Rail Station,Gas Station,Hotel Bar
9,E17,Pub,Café,Coffee Shop,Restaurant,Grocery Store,Pizza Place,Italian Restaurant,Fast Food Restaurant,Brewery,Bar


#### 3.5 Cluster Neighborhoods

In [23]:
# set number of clusters
kclusters = 12

london_grouped_clustering = london_grouped.drop('Venue Postal Outcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

london_merged = london_grouped[['Venue Postal Outcode']].copy()

# add clustering labels
london_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(neighborhoods_venues_sorted.set_index('Venue Postal Outcode'), on='Venue Postal Outcode')


#### 3.6 Examine results of clustering

In [24]:
london_merged[london_merged['Cluster Labels'] == 0]

,Venue Postal Outcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
121,SW1H,0,Hotel,Sandwich Place,Sporting Goods Shop,Restaurant,Café,Hotel Bar,Zoo Exhibit,Fish Market,Factory,Falafel Restaurant


In [25]:
london_merged[london_merged['Cluster Labels'] == 1]

,Venue Postal Outcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,IG8,1,Supermarket,Zoo Exhibit,Flower Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop


In [26]:
london_merged[london_merged['Cluster Labels'] == 2]

,Venue Postal Outcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,E10,2,Grocery Store,Park,Café,Coffee Shop,Pub,Hotel,Restaurant,Soccer Stadium,Train Station,Fast Food Restaurant
5,E13,2,Grocery Store,Metro Station,Pub,Bus Stop,Hostel,Gym / Fitness Center,Park,Bakery,Pharmacy,Fish & Chips Shop
14,E3,2,Pub,Café,Grocery Store,Bus Stop,Art Gallery,Coffee Shop,Park,Light Rail Station,Hotel,Historic Site
15,E4,2,Pub,Park,Italian Restaurant,Grocery Store,Fast Food Restaurant,Sandwich Place,Fish Market,Falafel Restaurant,Farm,Farmers Market
60,N20,2,Park,Cricket Ground,Pub,Brazilian Restaurant,Soccer Field,Golf Course,Zoo Exhibit,Fish Market,Farm,Farmers Market
71,NW10,2,Pub,Grocery Store,Plaza,Café,Bus Stop,Pharmacy,Middle Eastern Restaurant,Sandwich Place,Liquor Store,Coffee Shop
78,NW7,2,Sports Club,Farm,Park,Pub,Gastropub,Soccer Field,Athletics & Sports,Zoo Exhibit,Fish Market,Farmers Market
84,SE12,2,Café,Pub,Fast Food Restaurant,Coffee Shop,Soccer Field,Grocery Store,Rugby Pitch,Fish & Chips Shop,Italian Restaurant,Pet Store
95,SE22,2,Pub,Café,Park,Gastropub,Italian Restaurant,Grocery Store,Pizza Place,Ice Cream Shop,Indian Restaurant,Burger Joint
100,SE27,2,Pub,Grocery Store,Thrift / Vintage Store,Park,Breakfast Spot,Bakery,Train Station,Farmers Market,Pharmacy,Sandwich Place


In [27]:
london_merged[london_merged['Cluster Labels'] == 3]

,Venue Postal Outcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,E14,3,Coffee Shop,Italian Restaurant,Park,Burger Joint,Hotel,Gym / Fitness Center,Plaza,Trail,Sandwich Place,Café
13,E20,3,Café,Italian Restaurant,Toy / Game Store,Juice Bar,Grocery Store,Park,Pizza Place,Restaurant,Clothing Store,Sandwich Place
22,EC1M,3,Italian Restaurant,Pub,Hotel,Sandwich Place,French Restaurant,Nightclub,Museum,Steakhouse,Café,Fish & Chips Shop
28,EC2M,3,Hotel,Coffee Shop,Boxing Gym,Breakfast Spot,Historic Site,Hawaiian Restaurant,Salad Place,Colombian Restaurant,Gym / Fitness Center,Organic Grocery
41,EC4R,3,Trail,Wine Bar,Gym,Beer Bar,Seafood Restaurant,Asian Restaurant,Nightclub,Dessert Shop,Flower Shop,Farmers Market
58,N1C,3,Plaza,Train Station,Restaurant,Bookstore,Australian Restaurant,Modern European Restaurant,Movie Theater,Indian Restaurant,Lounge,Market
70,NW1,3,Zoo Exhibit,Coffee Shop,Pub,Hotel,Café,Japanese Restaurant,Museum,Music Venue,Park,Theater
81,SE1,3,Pub,Hotel,Coffee Shop,Restaurant,Café,Theater,Scenic Lookout,Art Gallery,Cocktail Bar,Italian Restaurant
109,SW10,3,Restaurant,Italian Restaurant,Bakery,Grocery Store,Sandwich Place,Pub,Diner,Peruvian Restaurant,Burger Joint,Sports Bar
118,SW19,3,Coffee Shop,Italian Restaurant,Pub,Sushi Restaurant,Grocery Store,Café,Supermarket,Restaurant,Pizza Place,Bar


In [28]:
london_merged[london_merged['Cluster Labels'] == 4]

,Venue Postal Outcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,E12,4,Gym / Fitness Center,Fried Chicken Joint,Train Station,Pub,Indian Restaurant,Restaurant,Park,Turkish Restaurant,Film Studio,Exhibit
10,E18,4,Italian Restaurant,Café,Bar,Grocery Store,Coffee Shop,Furniture / Home Store,Supermarket,Indian Restaurant,Thai Restaurant,Bus Stop
17,E6,4,Grocery Store,Pub,Coffee Shop,Pizza Place,Park,Toy / Game Store,Gym / Fitness Center,Shopping Plaza,Electronics Store,Sandwich Place
18,E7,4,Fast Food Restaurant,Indian Restaurant,Grocery Store,Pub,Train Station,Italian Restaurant,Comfort Food Restaurant,Café,Fish & Chips Shop,Shopping Mall
49,N11,4,Grocery Store,Coffee Shop,Bus Stop,Sporting Goods Shop,Café,Furniture / Home Store,Breakfast Spot,Fast Food Restaurant,Park,Middle Eastern Restaurant
50,N12,4,Coffee Shop,Café,Supermarket,Grocery Store,Bookstore,Breakfast Spot,Mobile Phone Shop,Fast Food Restaurant,Soccer Field,Sushi Restaurant
51,N13,4,Pub,Grocery Store,Park,Bar,Fast Food Restaurant,Supermarket,Bookstore,Coffee Shop,Greek Restaurant,Italian Restaurant
52,N14,4,Gym / Fitness Center,Park,Fast Food Restaurant,Coffee Shop,Café,Steakhouse,Supermarket,Burger Joint,Metro Station,Pharmacy
53,N15,4,Coffee Shop,Grocery Store,Café,Park,Clothing Store,Pizza Place,Pharmacy,Chinese Restaurant,Mobile Phone Shop,Performing Arts Venue
55,N17,4,Café,Bus Stop,Grocery Store,Bus Station,Pizza Place,Brewery,Gym,Scenic Lookout,Bar,Train Station


In [29]:
london_merged[london_merged['Cluster Labels'] == 5]

,Venue Postal Outcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,IG1,5,Supermarket,Grocery Store,Zoo Exhibit,Flower Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio


In [30]:
london_merged[london_merged['Cluster Labels'] == 6]

,Venue Postal Outcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
101,SE28,6,Pet Store,Pharmacy,Gym / Fitness Center,Supermarket,Clothing Store,Fast Food Restaurant,Fish & Chips Shop,Factory,Falafel Restaurant,Farm


In [31]:
london_merged[london_merged['Cluster Labels'] == 7]

,Venue Postal Outcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,E1,7,Pub,Coffee Shop,Hotel,Indian Restaurant,Pizza Place,Café,Grocery Store,Turkish Restaurant,Park,Thai Restaurant
3,E11,7,Pub,Café,Coffee Shop,Pharmacy,Bakery,Restaurant,Fish & Chips Shop,Fast Food Restaurant,Metro Station,Supermarket
7,E15,7,Pub,Café,Hotel,Grocery Store,General Entertainment,Bus Stop,Bar,Park,Coffee Shop,Fast Food Restaurant
9,E17,7,Pub,Café,Coffee Shop,Restaurant,Grocery Store,Pizza Place,Italian Restaurant,Fast Food Restaurant,Brewery,Bar
11,E1W,7,Pub,Coffee Shop,Park,Restaurant,Italian Restaurant,Breakfast Spot,Street Food Gathering,Supermarket,Tapas Restaurant,Event Space
12,E2,7,Pub,Coffee Shop,Café,Wine Bar,Italian Restaurant,Park,Restaurant,Art Gallery,Cocktail Bar,Pizza Place
16,E5,7,Pub,Café,Grocery Store,Nature Preserve,Coffee Shop,Turkish Restaurant,Breakfast Spot,Garden,Chinese Restaurant,Bakery
19,E8,7,Café,Coffee Shop,Pub,Bakery,Restaurant,Breakfast Spot,Cocktail Bar,Bookstore,Yoga Studio,Vietnamese Restaurant
20,E9,7,Pub,Café,Bar,Coffee Shop,Brewery,Italian Restaurant,Vegetarian / Vegan Restaurant,Food & Drink Shop,Gastropub,Middle Eastern Restaurant
23,EC1N,7,Coffee Shop,Pub,Food Truck,Falafel Restaurant,Gym / Fitness Center,Italian Restaurant,Vegetarian / Vegan Restaurant,Burrito Place,Flea Market,Tea Room


In [32]:
london_merged[london_merged['Cluster Labels'] == 8]

,Venue Postal Outcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,CR4,8,Train Station,Indian Restaurant,Fish & Chips Shop,Coffee Shop,Platform,Fish Market,Factory,Falafel Restaurant,Farm,Farmers Market
21,EC1A,8,Coffee Shop,Gym / Fitness Center,Church,Modern European Restaurant,Vietnamese Restaurant,English Restaurant,History Museum,Garden,Park,Bar
26,EC1Y,8,Coffee Shop,Gym / Fitness Center,Food Truck,Turkish Restaurant,Café,Movie Theater,Burger Joint,Salad Place,Vietnamese Restaurant,Sushi Restaurant
31,EC2V,8,Coffee Shop,Italian Restaurant,Art Gallery,Mexican Restaurant,Modern European Restaurant,Cocktail Bar,Bookstore,Vietnamese Restaurant,Steakhouse,Food Stand
32,EC2Y,8,Art Gallery,Botanical Garden,Performing Arts Venue,Theater,Indie Movie Theater,Scenic Lookout,Concert Hall,Coffee Shop,Fish Market,Farm
33,EC3A,8,Salad Place,Cupcake Shop,Steakhouse,Cocktail Bar,Gym / Fitness Center,Restaurant,Street Food Gathering,Mini Golf,Japanese Restaurant,Coffee Shop
34,EC3M,8,Coffee Shop,Steakhouse,Office,Portuguese Restaurant,Thai Restaurant,Scenic Lookout,Cocktail Bar,Zoo Exhibit,Falafel Restaurant,Farm
39,EC4M,8,Coffee Shop,Sandwich Place,Scenic Lookout,Restaurant,Burger Joint,Butcher,Steakhouse,Falafel Restaurant,Tea Room,Seafood Restaurant
40,EC4N,8,Coffee Shop,Italian Restaurant,Wine Bar,Gym / Fitness Center,Seafood Restaurant,Udon Restaurant,History Museum,French Restaurant,Indian Restaurant,Design Studio
42,EC4V,8,Coffee Shop,Café,Sandwich Place,Modern European Restaurant,Gym / Fitness Center,Vietnamese Restaurant,Hotel,South American Restaurant,Italian Restaurant,Indian Restaurant


In [33]:
london_merged[london_merged['Cluster Labels'] == 9]

,Venue Postal Outcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
136,TW9,9,Seafood Restaurant,Bus Stop,Park,Soccer Field,Zoo Exhibit,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [34]:
london_merged[london_merged['Cluster Labels'] == 10]

,Venue Postal Outcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,KT3,10,Bus Stop,French Restaurant,Bakery,Zoo Exhibit,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio


In [35]:
london_merged[london_merged['Cluster Labels'] == 11]

,Venue Postal Outcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,E16,11,Hotel,Coffee Shop,Chinese Restaurant,Bar,Harbor / Marina,Café,Sandwich Place,Light Rail Station,Gas Station,Hotel Bar
27,EC2A,11,Hotel,Coffee Shop,Bar,Cocktail Bar,Sandwich Place,Food Truck,Street Food Gathering,Gym / Fitness Center,Gym,Beer Garden
30,EC2R,11,Hotel,Seafood Restaurant,Coffee Shop,Yoga Studio,French Restaurant,Steakhouse,Lounge,Factory,Falafel Restaurant,Farm
35,EC3N,11,Hotel,French Restaurant,Hotel Bar,Castle,Park,Scenic Lookout,History Museum,General Entertainment,Gym / Fitness Center,Exhibit
36,EC3R,11,Hotel,Italian Restaurant,Historic Site,Restaurant,Coffee Shop,Park,South American Restaurant,Burger Joint,Garden,Museum
43,EC4Y,11,Building,Wine Bar,Noodle House,Beer Bar,Coffee Shop,Distillery,Hotel,Garden,Fountain,Frozen Yogurt Shop
119,SW1A,11,Plaza,Monument / Landmark,Hotel,Historic Site,Palace,Hotel Bar,Spa,Theater,Mexican Restaurant,Park
120,SW1E,11,Coffee Shop,Theater,Hotel,Sushi Restaurant,Gym,Historic Site,Gift Shop,Chocolate Shop,Scandinavian Restaurant,Supermarket
123,SW1V,11,Hotel,Italian Restaurant,Café,Sandwich Place,Pub,Park,Bakery,Cheese Shop,Turkish Restaurant,Chinese Restaurant
125,SW1X,11,Hotel,Boutique,Café,Restaurant,Gastropub,Shoe Store,Cocktail Bar,French Restaurant,Garden,Italian Restaurant


### Mapping

#### 4.0 Calculate the number of rows very close and fairly close to our postcode centroids

In [38]:
cols=['Postcode','Very Close','Fairly Close','Total']
results = pd.DataFrame(columns=cols)

for i in london_postcodes['Postcode']:
    veryClose = london_venues[london_venues[i] <= 250].shape[0]
    fairlyClose = london_venues[(london_venues[i] > 250) & (london_venues[i] < 1000)].shape[0]
    total = london_venues[(london_venues[i] <= 1000)].shape[0]
    tmpResults = pd.DataFrame([[i, veryClose, fairlyClose, total]]).reset_index(drop=True)
    tmpResults.columns=cols
    #print(tmpResults)
    results = pd.concat([results, tmpResults],axis=0)

#### 4.1 Let's see which postcode 

In [40]:
results[results['Very Close'] > 47]

,Postcode,Very Close,Fairly Close,Total
0,W1F,48,365,413
0,WC2E,52,271,323


#### 4.2 Let see if we have other locations that are higher than 270 fairly close

In [42]:
results[results['Total'] >= 400]

,Postcode,Very Close,Fairly Close,Total
0,W1D,46,370,416
0,W1F,48,365,413


#### 4.3 Let's look for some particularly bad choices

In [45]:
results[(results['Very Close'] < 2) & (results['Total'] < 20)]

,Postcode,Very Close,Fairly Close,Total
0,E4,0,9,9
0,E6,1,14,15
0,E12,1,11,12
0,N20,0,5,5
0,N21,1,17,18
0,NW7,0,7,7
0,SE2,1,7,8
0,SE9,0,6,6
0,SE12,0,18,18
0,SE18,0,15,15


#### 4.4 Let's take a look at the postcode W1D

In [51]:
london_postcodes[london_postcodes['Postcode'] == 'W1F']

,Postal Region,Postcode,Post town,Coverage,Local authority area,Latitude,Longitude
142,W,W1F,LONDON,Soho (north west),Westminster,51.514272,-0.135817


#### 4.5 W1F the first of our postcodes with the highest closeby count

In [57]:
postcode = 'W1F'
latitude = london_postcodes[london_postcodes['Postcode'] == postcode]['Latitude'].values[0]
longitude = london_postcodes[london_postcodes['Postcode'] == postcode]['Longitude'].values[0]

print(latitude, longitude)
cutDown = london_venues[london_venues[postcode] <= 250].reset_index(drop=True)

map_london = folium.Map(location=[latitude, longitude], zoom_start=14)

folium.Circle(
    [latitude, longitude],
    radius=1000,
    color='#008000',
    fill=True,
    fill_color='#ADFF2F',
    fill_opacity=0.3,
    parse_html=False).add_to(map_london) 

# Highlight the 250m radius
folium.Circle(
    [latitude, longitude],
    radius=250,
    color='#7B68EE',
    fill=True,
    fill_color='#87CEFA',
    fill_opacity=0.4,
    parse_html=False).add_to(map_london)  

# add markers showing close to postcode centroid
for lat, lng, category, venue in zip(cutDown['Venue Latitude']
                                           , cutDown['Venue Longitude']
                                           , cutDown['Venue Category']
                                           , cutDown['Venue']):
    label = '{}, {}'.format(venue, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#008000',
        fill=True,
        fill_color='#87CEFA',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
cutDown = london_venues[(london_venues[postcode] > 250) & (london_venues[postcode] <= 1000)].reset_index(drop=True)
    
# add markers showing close to postcode centroid
for lat, lng, category, venue in zip(cutDown['Venue Latitude']
                                           , cutDown['Venue Longitude']
                                           , cutDown['Venue Category']
                                           , cutDown['Venue']):
    label = '{}, {}'.format(venue, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#008000',
        fill=True,
        fill_color='#90EE90',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
 
    
map_london 

51.5142722 -0.1358166


### 3.2 WC2E the second of our postcodes with the highest closeby count

In [49]:
postcode = 'WC2E'

latitude = london_postcodes[london_postcodes['Postcode'] == postcode]['Latitude'].values[0]
longitude = london_postcodes[london_postcodes['Postcode'] == postcode]['Longitude'].values[0]

print(latitude, longitude)
cutDown = london_venues[london_venues[postcode] <= 250].reset_index(drop=True)

map_london = folium.Map(location=[latitude, longitude], zoom_start=14)



folium.Circle(
    [latitude, longitude],
    radius=1000,
    color='#008000',
    fill=True,
    fill_color='#ADFF2F',
    fill_opacity=0.3,
    parse_html=False).add_to(map_london) 

# Highlight the 250m radius
folium.Circle(
    [latitude, longitude],
    radius=250,
    color='#7B68EE',
    fill=True,
    fill_color='#87CEFA',
    fill_opacity=0.4,
    parse_html=False).add_to(map_london)  

# add markers showing close to postcode centroid
for lat, lng, category, venue in zip(cutDown['Venue Latitude']
                                           , cutDown['Venue Longitude']
                                           , cutDown['Venue Category']
                                           , cutDown['Venue']):
    label = '{}, {}'.format(venue, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#008000',
        fill=True,
        fill_color='#87CEFA',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
cutDown = london_venues[(london_venues[postcode] > 250) & (london_venues[postcode] <= 1000)].reset_index(drop=True)
    
# add markers showing close to postcode centroid
for lat, lng, category, venue in zip(cutDown['Venue Latitude']
                                           , cutDown['Venue Longitude']
                                           , cutDown['Venue Category']
                                           , cutDown['Venue']):
    label = '{}, {}'.format(venue, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#008000',
        fill=True,
        fill_color='#90EE90',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
 
    
map_london 

51.51166019999999 -0.1211039


### 3.3 W1F our postcode with the highest closeby plus nearby count

In [58]:
postcode = 'W1F'

latitude = london_postcodes[london_postcodes['Postcode'] == postcode]['Latitude'].values[0]
longitude = london_postcodes[london_postcodes['Postcode'] == postcode]['Longitude'].values[0]

print(latitude, longitude)
cutDown = london_venues[london_venues[postcode] <= 250].reset_index(drop=True)

map_london = folium.Map(location=[latitude, longitude], zoom_start=14)



folium.Circle(
    [latitude, longitude],
    radius=1000,
    color='#008000',
    fill=True,
    fill_color='#ADFF2F',
    fill_opacity=0.3,
    parse_html=False).add_to(map_london) 

# Highlight the 250m radius
folium.Circle(
    [latitude, longitude],
    radius=250,
    color='#7B68EE',
    fill=True,
    fill_color='#87CEFA',
    fill_opacity=0.4,
    parse_html=False).add_to(map_london)  

# add markers showing close to postcode centroid
for lat, lng, category, venue in zip(cutDown['Venue Latitude']
                                           , cutDown['Venue Longitude']
                                           , cutDown['Venue Category']
                                           , cutDown['Venue']):
    label = '{}, {}'.format(venue, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#008000',
        fill=True,
        fill_color='#87CEFA',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
cutDown = london_venues[(london_venues[postcode] > 250) & (london_venues[postcode] <= 1000)].reset_index(drop=True)
    
# add markers showing close to postcode centroid
for lat, lng, category, venue in zip(cutDown['Venue Latitude']
                                           , cutDown['Venue Longitude']
                                           , cutDown['Venue Category']
                                           , cutDown['Venue']):
    label = '{}, {}'.format(venue, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#008000',
        fill=True,
        fill_color='#90EE90',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
 
    
map_london 

51.5142722 -0.1358166


### 3.4 NW7 and example of a postcode with the low venue count

In [54]:
postcode = 'NW7'

latitude = london_postcodes[london_postcodes['Postcode'] == postcode]['Latitude'].values[0]
longitude = london_postcodes[london_postcodes['Postcode'] == postcode]['Longitude'].values[0]

print(latitude, longitude)
cutDown = london_venues[london_venues[postcode] <= 250].reset_index(drop=True)

map_london = folium.Map(location=[latitude, longitude], zoom_start=14)



folium.Circle(
    [latitude, longitude],
    radius=1000,
    color='#008000',
    fill=True,
    fill_color='#ADFF2F',
    fill_opacity=0.3,
    parse_html=False).add_to(map_london) 

# Highlight the 250m radius
folium.Circle(
    [latitude, longitude],
    radius=250,
    color='#7B68EE',
    fill=True,
    fill_color='#87CEFA',
    fill_opacity=0.4,
    parse_html=False).add_to(map_london)  

# add markers showing close to postcode centroid
for lat, lng, category, venue in zip(cutDown['Venue Latitude']
                                           , cutDown['Venue Longitude']
                                           , cutDown['Venue Category']
                                           , cutDown['Venue']):
    label = '{}, {}'.format(venue, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#008000',
        fill=True,
        fill_color='#87CEFA',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
cutDown = london_venues[(london_venues[postcode] > 250) & (london_venues[postcode] <= 1000)].reset_index(drop=True)
    
# add markers showing close to postcode centroid
for lat, lng, category, venue in zip(cutDown['Venue Latitude']
                                           , cutDown['Venue Longitude']
                                           , cutDown['Venue Category']
                                           , cutDown['Venue']):
    label = '{}, {}'.format(venue, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#008000',
        fill=True,
        fill_color='#90EE90',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
 
    
map_london 

51.61473059999999 -0.230102


In [60]:
neighborhoods_venues_sorted[neighborhoods_venues_sorted['Venue Postal Outcode'].isin(['W1F','W1D','WC2E'])]

,Venue Postal Outcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
145,W1D,Theater,Hotel,Ice Cream Shop,Liquor Store,Lounge,Park,Plaza,Tapas Restaurant,Steakhouse,Sri Lankan Restaurant
146,W1F,Cocktail Bar,Coffee Shop,Italian Restaurant,Art Gallery,Burger Joint,BBQ Joint,Chocolate Shop,Cupcake Shop,Tapas Restaurant,Lingerie Store
173,WC2E,Dessert Shop,Italian Restaurant,Burger Joint,French Restaurant,Clothing Store,Pub,Theater,Cupcake Shop,Sushi Restaurant,Boutique
